## Basic codes

The purpose of this document is to give a condensed presentation of the various codes used to implement the ablation model, uniform or not, in one or two dimensions. For the sake of brevity, no examples will be shown here, just the raw codes, with as much commentary as possible. 

### 1D uniform ablation

#### First version

Simplest implementation of the model. The interface is represented by a set of points $(x,z)$. The algorithm takes as input the list of abscissas ($\textit{xx}$) and the list of ordinates ($\textit{zz}$), as well as the distance $\textit{l}$ by which the interface should be eroded. It returns the list of abscissas and the list of ordinates corresponding to the interface obtained after erosion.

First, the algorithm calculates the normal vector, pointing downwards, at each point of the interface. It then propagates each point of the interface by a distance $\textit{l}$ along this normal vector to obtain the new interface. However, if we stop there, we see "loops" or "swallow-tails" appear at the interface points where the radius of curvature is less than $\textit{l}$. These loops need to be removed to obtain the 'physical' interface: that's what the antepenultimate line of code does. The idea is that deleting these loops means deleting all the points located at a distance strictly less than $\textit{l}$ from the initial interface (i.e. located at a distance strictly less than $\textit{l}$ from at least one of the points on the initial interface).

In concrete terms, $\textit{distance_matrix(new_coord, ex_coord)}$ returns a matrix $D_{ij}$ containing the distances between each point on the new interface (index $i$) and each point on the old interface (index $j$). $\textit{distance_matrix(new_coord,ex_coord)<l*se}$ is a matrix whose index $D^{'}_{ij}$ takes the value $\textit{True}$ if the distance between the point $i$ of the new interface and the point $j$ of the old interface are closer than $\textit{l}\cdot \textit{se}$, $\textit{False}$ otherwise. The distances are compared to $\textit{l}\cdot \textit{se}$ rather than $\textit{l}$ for reasons of rounding: if you don't do this, the code sometimes considers that a point on the new interface is strictly closer than $\textit{l}$ to the point on the old interface from which it originated (even though it is theoretically at a distance equal to $\textit{l}$, since this is how it was constructed); this leads to the deletion of the said point. Taking $\textit{se}$ very close to 1 but strictly less than 1 avoids this inconvenience. Next, $\textit{np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])}$ is a list containing the index $i$ if there is at least one $j$ such that $D^{'}_{ij}$ has the value $\textit{True}$. Finally, $\textit{np.delete(new_coord,np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])),axis=0)}$ deletes all the points whose indices are in the previous list. In this way, the loops have been eliminated and the algorithm ends up returning the list of abscissas and the list of ordinates of the new eroded interface.   

In [1]:
se=0.999999
def erod(xx,zz,l):
    #points obtained by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])#computation of a normal vector
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)#normal vector of norm l
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]#new points obtained     
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop,zzprop]))
    ex_coord=np.transpose(np.array([xx,zz]))
    cleaned_new_coord=np.delete(new_coord,np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])),axis=0)
    final_xx,final_zz=cleaned_new_coord[:,0],cleaned_new_coord[:,1]
    return final_xx, final_zz

#### Improved version

The version of the algorithm presented above has the disadvantage of being fairly costly in terms of calculation time, particularly because of the $\textit{distance_matrix}$ which quickly becomes cumbersome to calculate as the number of interface points increases. However, this matrix calculates many unnecessary distances: it is clear that it is not necessary to calculate the distance between the leftmost point of the initial interface and the rightmost point of the eroded interface to know that it is greater than $\textit{l}$. The version presented below thus improves the algorithm by calculating, for a point on the new interface, only its distance from the $N_v$ neighbours to the right and left of the point on the initial interface from which it originated. We have therefore replaced the calculation of $\textit{distance_matrix}$, of size $n \cdot n$ with $n$ the number of points, by the calculation of the matrix $\textit{distance_neighbours}$ of size $n \cdot N_v$.

$N_v$ is a parameter that must be chosen carefully. If $N_v$ is too small, you run the risk of keeping points that should be removed (points belonging to the loops we talked about earlier); but if $N_v$ is too big, then the code is too slow. A good way of doing this (in my opinion) is to choose $N_v$ as follows: $N_v=2l/\mathrm{dx}$ where $\mathrm{dx}$ is the distance along the x axis between two successive points on the initial interface (this is possible if $\mathrm{dx}$ is constant). By determining $N_v$ in this way, we can be sure to remove all the points that need to be removed (see triangular inequality).

In [1]:
def distance_neighbours(ex,new,n):
    distmat=np.zeros((2*n+1,np.shape(new)[1]))
    for k in range(-n,n+1):
        ex_rollk=np.roll(ex,k)
        distmat[n+k]=np.sqrt(np.sum((new-ex_rollk)**2,axis=0))
    return np.concatenate((distmat[:n],distmat[n+1:]),axis=0)   

def erod_improved(xx,zz,l,n):
    #points obtain by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])# Calcul du vecteur normal initial.
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]    
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.array([xxprop,zzprop])
    ex_coord=np.array([xx,zz])
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_neighbours(ex_coord,new_coord,n)<l*se).nonzero()[1]),axis=1)
    final_xx,final_zz=cleaned_new_coord[0,:],cleaned_new_coord[1,:]
    return final_xx, final_zz

### 1D non-uniform ablation

The algorithm below is a generalisation of the previous algorithm. Ablation is no longer necessarily uniform. The erosion distance, which was previously the same everywhere (it was denoted $\textit{l}$), is now a priori different for each point ($\textit{l}=\textit{l(i)}$). In concrete terms, instead of multiplying the list of normal vectors by a scalar $\textit{l}$, we multiply it term by term with the list $\textit{l} \cdot f_{er}(xx,zz,e)$, where $f_{er}(xx,zz,e)$ is a function which assigns an erosion distance to each point on the interface, depending a priori on the shape of the interface and on an additional parameter $pp$. This function is entered as code input by the user, so you can imagine all sorts of more or less twisted things.

The removal of loops in the second part of the code is slightly more subtle than in the uniform case. To keep a point, it is no longer enough to check that it is strictly further than $\textit{l}$ from the initial interface; it must be further than $l(i)$ from each point $i$ of the initial interface, which requires the code to be modified.

In [ ]:
def erod_non_uniform_improved(xx,zz,l,f_er,pp,n):
    ff=f_er(xx,zz,pp)
    #points obtain by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])
    vecn=l*ff*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]    
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.array([xxprop,zzprop])
    ex_coord=np.array([xx,zz])
    f_er_mat=np.array([np.concatenate((np.roll(ff,n-i)[:n],np.roll(ff,n-i)[n+1:2*n+1])) for i in range(len(xxprop))]).transpose()
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_neighbours(ex_coord,new_coord,n)<l*se*f_er_mat).nonzero()[1]),axis=1)
    final_xx,final_zz=cleaned_new_coord[0,:],cleaned_new_coord[1,:]
    return final_xx, final_zz

### 2D uniform and non-uniform ablation

The algorithms below are extensions of the 1D ablation algorithm to 2D: the interface is now a surface ($z(x,y)$). The principle of the algorithm is broadly similar to the principle of the $\textit{erod_improved}$ function. To remove the loops, we look at the distance between each point on the new interface and the neighbours ($nx$ in the x direction, $ny$ in the y direction) of the point on the initial interface from which that point originated. All these distances are contained in the three-dimensional array returned by the function $\textit{distance_neighbours_2d}$. 

In concrete terms, the initial surface is a set of points located on a grid, which can be reconstructed from the two lists $xx$ and $yy$ corresponding to the coordinates of the grid points in the $x$ and $y$ directions respectively. The algorithm takes as input these two lists and the matrix $zz$ corresponding to the heights of the points on the said grid. It also takes as input the erosion distance $l$ and the number of neighbours $nx$ and $ny$ on which to make the comparison. 

At the end, the code returns the matrix $zer$ corresponding to the heights of the eroded interface on the same grid as the input grid; to do this, it performs a Delaunay interpolation. 

In [3]:
def distance_neighbours_2d(ex,new,dx,dy,nx,ny):
    distmat=np.zeros((2*nx+1,2*ny+1,len(ex)))
    for k in range(-nx,nx+1):
        for j in range(-ny,ny+1):
            ex_roll_kj=np.roll(ex,-dx*j-k,axis=0)
            distmat[nx+k,ny+j]=np.sqrt(np.sum((new-ex_roll_kj)**2,axis=1))
    return distmat

def propag_neighbours_2d(ff,lx,ly,nx,ny):
    neighbmat=np.zeros((2*nx+1,2*ny+1,len(ff)))
    for k in range(-nx,nx+1):
        for j in range(-ny,ny+1):
            ff_roll_kj=np.roll(ff,-lx*j-k)
            voismat[nx+k,ny+j]=ff_roll_kj
    return neighbmat   

def erod_improved_2d(xx,yy,zz,nx,ny,l):
    #initialization
    dx,dy=len(xx),len(yy)
    XX,YY=np.meshgrid(xx,yy)
    #points obtain by moving by a distance l along the normal
    gradvec=np.append(np.gradient(zz.transpose(),xx,yy),[0*zzt-1],axis=0)#normal vector
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=XX+vecn[0,:].transpose()
    yyprop=YY+vecn[1,:].transpose()
    zzprop=zz+vecn[2,:].transpose()
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop.flatten(),yyprop.flatten(),zzprop.flatten()]))
    ex_coord=np.transpose(np.array([XX.flatten(),YY.flatten(),zz.flatten()]))
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_neighbours_2d(ex_coord,new_coord,dx,dy,nx,ny)<l*se).nonzero()[2]),axis=0)
    #return points on a regular grid by interpolating
    interpolator = tri.LinearTriInterpolator(tri.Triangulation(cleaned_new_coord[:,0],cleaned_new_coord[:,1]), cleaned_new_coord[:,2])
    return interpolator(XX, YY)

def erod_nonuniform_improved_2d(xx,yy,zz,nx,ny,l,fer,pp):
    #initialization
    lx,ly=len(xx),len(yy)
    XX,YY=np.meshgrid(xx,yy)
    #points obtain by moving by a distance l along the normal
    gradvec=np.append(np.gradient(zz.transpose(),xx,yy),[0*zz.transpose()-1],axis=0)#normal vector 
    vecn=l*fer(XX,YY,zz,pp).transpose()*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=XX+vecn[0,:].transpose()
    yyprop=YY+vecn[1,:].transpose()
    zzprop=zz+vecn[2,:].transpose()
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop.flatten(),yyprop.flatten(),zzprop.flatten()]))
    ex_coord=np.transpose(np.array([XX.flatten(),YY.flatten(),zz.flatten()]))
    ff=fer(XX,YY,zz,pp).flatten()
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_neighbours_2d(ex_coord,new_coord,lx,ly,nx,ny)<l*se*propag_neighbours_2d(ff,lx,ly,nx,ny)).nonzero()[2]),axis=0)
    #return points on a regular grid by interpolating
    interpolator = tri.LinearTriInterpolator(tri.Triangulation(cleaned_new_coord[:,0],cleaned_new_coord[:,1]), cleaned_new_coord[:,2])
    return interpolator(XX, YY)